In [303]:
import random
import itertools
import numpy as np

In [304]:
def generate_game(bigNumbers) -> tuple[list[int],int]:
    if bigNumbers>4 or bigNumbers<0:
        raise ValueError("Invalid bigNumbers")

    largeNumbers=random.sample([25,50,75,100],bigNumbers)
    smallNumbers=random.sample(list(range(1,11))+list(range(1,11)),6-bigNumbers)

    numbers = largeNumbers + smallNumbers

    return (numbers, random.randint(101, 999))


In [305]:
operations = {
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
    "*": lambda x, y: x * y,
    "/": lambda x, y: x // y,  # integer division
}


def recusive_solve_countdown(numbers, target, calculations=[]):

    solutions = []


    # iterate over all pair of number pairs
    for pair in itertools.combinations(numbers, 2):

        # Remove the pair from the list of numbers and make copy
        computedNumbers = numbers.copy()
        computedNumbers.remove(pair[0])
        computedNumbers.remove(pair[1])

        for op, calculate in operations.items():
            a = max(pair)
            b = min(pair)

            val = calculate(a, b)
            print(calculations,a, op, b, "=", val, val == target)

            calcs = [*calculations, (a, op, b)]

            if val == target:
                solutions.append(calcs)

            sols = recusive_solve_countdown([val, *computedNumbers], target, calcs)
            if len(sols) != 0:
                solutions=[*sols, *solutions]

        # print(pair,computedNumbers)

    return solutions

In [306]:
def solve_numbers(game):
    numbers = game[0]
    target  = game[1]

    # # Check for invalid inputs
    # if len(numbers) != 6:
    #     raise ValueError("Invalid number of numbers")

    # if not 101 <= target <= 999:
    #     raise ValueError("Invalid target number")

    return recusive_solve_countdown(numbers, target)


In [307]:
game = generate_game(2)

testGame = ([5,2,15], 5)

solve_numbers(testGame)

[] 5 + 2 = 7 False
[(5, '+', 2)] 15 + 7 = 22 False
[(5, '+', 2)] 15 - 7 = 8 False
[(5, '+', 2)] 15 * 7 = 105 False
[(5, '+', 2)] 15 / 7 = 2 False
[] 5 - 2 = 3 False
[(5, '-', 2)] 15 + 3 = 18 False
[(5, '-', 2)] 15 - 3 = 12 False
[(5, '-', 2)] 15 * 3 = 45 False
[(5, '-', 2)] 15 / 3 = 5 True
[] 5 * 2 = 10 False
[(5, '*', 2)] 15 + 10 = 25 False
[(5, '*', 2)] 15 - 10 = 5 True
[(5, '*', 2)] 15 * 10 = 150 False
[(5, '*', 2)] 15 / 10 = 1 False
[] 5 / 2 = 2 False
[(5, '/', 2)] 15 + 2 = 17 False
[(5, '/', 2)] 15 - 2 = 13 False
[(5, '/', 2)] 15 * 2 = 30 False
[(5, '/', 2)] 15 / 2 = 7 False
[] 15 + 5 = 20 False
[(15, '+', 5)] 20 + 2 = 22 False
[(15, '+', 5)] 20 - 2 = 18 False
[(15, '+', 5)] 20 * 2 = 40 False
[(15, '+', 5)] 20 / 2 = 10 False
[] 15 - 5 = 10 False
[(15, '-', 5)] 10 + 2 = 12 False
[(15, '-', 5)] 10 - 2 = 8 False
[(15, '-', 5)] 10 * 2 = 20 False
[(15, '-', 5)] 10 / 2 = 5 True
[] 15 * 5 = 75 False
[(15, '*', 5)] 75 + 2 = 77 False
[(15, '*', 5)] 75 - 2 = 73 False
[(15, '*', 5)] 75 * 2 =

[[(15, '/', 5), (3, '+', 2)],
 [(15, '-', 5), (10, '/', 2)],
 [(5, '*', 2), (15, '-', 10)],
 [(5, '-', 2), (15, '/', 3)]]